# Mac Camera Walkthrough
>  Documentation 



In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev.showdoc import *
from fastcore.test import *


In [ ]:
from UAV.mavlink.camera import CameraClient, CameraServer, MAVCom, boot_time_str
from UAV.mavlink.component import Component, mavutil, mavlink
from UAV.camera.fake_cam import CV2Camera, GSTCamera, read_camera_dict_from_toml
from gstreamer import  GstPipeline
import gstreamer.utils as gst_utils
import time
from pathlib import Path
import cv2

In [ ]:
MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA
# cli = GimbalClient(mav_connection=None, source_component=11, mav_type=MAV_TYPE_GCS, debug=False)
# gim1 = GimbalServer(mav_connection=None, source_component=22, mav_type=MAV_TYPE_CAMERA, debug=False)
SINK_PIPELINE = gst_utils.to_gst_string([
            'udpsrc port=5000 ! application/x-rtp, media=(string)video, clock-rate=(int)90000, encoding-name=(string)H264, payload=(int)96',
            'rtph264depay ! avdec_h264',
            'fpsdisplaysink',
            # 'autovideosink',
        ])
con1, con2 = "udpin:localhost:14445", "udpout:localhost:14445"
# con1, con2 = "/dev/ttyACM0", "/dev/ttyUSB0"

if __name__ == '__main__':
    print (f"{boot_time_str =}")
    config_path = Path("../../config")

    cam_gst_1 = GSTCamera(camera_dict=read_camera_dict_from_toml(config_path / "test_camera_info.toml"))
    cam_cv2_1 = CV2Camera(camera_dict=read_camera_dict_from_toml(config_path / "test_camera_info.toml"))
    with GstPipeline(SINK_PIPELINE) as rcv_pipeline: # this will show the video on fpsdisplaysink
        with MAVCom(con1, source_system=111, debug=False) as client:
            with MAVCom(con2, source_system=222, debug=False) as server:
                cam:CameraClient = client.add_component(
                    CameraClient(mav_type=MAV_TYPE_GCS, source_component=11, debug=False))
                server.add_component(CameraServer(mav_type=MAV_TYPE_CAMERA, source_component=22, camera=cam_gst_1, debug=False))
                # server.add_component(CameraServer(mav_type=MAV_TYPE_CAMERA, source_component=22, camera=None, debug=False))

                cam.wait_heartbeat(target_system=222, target_component=22, timeout=1)
                time.sleep(0.1)
                cam.set_target(222, 22)


                cam.request_camera_information()
                time.sleep(0.1)

                cam.request_storage_information()

                cam.request_camera_capture_status()

                cam.request_camera_information()

                cam.request_camera_settings()

                cam.image_start_capture(interval=0.1, count=10)

                while cam_gst_1.capture_thread.is_alive():
                    if cam_gst_1.last_image is not None:
                        cv2.imshow('gst_src', cam_gst_1.last_image)
                        cam_gst_1.last_image = None
                    cv2.waitKey(10)

                # time.sleep(2)
                start = time.time()
                msg = cam.request_storage_information()
                print (msg)
                print(f"request_storage_information time : {1000*(time.time() - start) = } msec")
                
    cv2.destroyAllWindows()


INFO   | uav.GSTCamera   | 42.416 | fake_cam.py:127 | MainThread         | GSTCamera Started
INFO   | pygst.GstVidSrc | 42.417 | gst_tools.py:131 | MainThread         | GstVidSrcValve 
 gst-launch-1.0 videotestsrc pattern=ball flip=true is-live=true num-buffers=1000 ! video/x-raw,framerate=10/1 !  tee name=t t. ! queue leaky=2 ! valve name=myvalve drop=False ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc tune=zerolatency ! rtph264pay ! udpsink host=127.0.0.1 port=5000 t. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=5/1,format=(string)BGR ! videoconvert ! appsink name=mysink emit-signals=true  sync=false async=false  max-buffers=2 drop=true 
INFO   | pygst.GstVidSrc | 42.420 | gst_tools.py:193 | MainThread         | Starting GstVidSrcValve
DEBUG  | pygst.GstVidSrc | 42.421 | gst_tools.py:197 | MainThread         | GstVidSrcValve Setting pipeline state to PLAYING ... 
DEBUG  | pygst.GstVidSrc | 42.421 | gst_tools.py:203 | Main

boot_time_str ='2023-09-16|11:16:42'


INFO   | uav.MAVCom      | 42.630 |  mavcom.py:393 | Thread-18 (listen) | MAVLink Mav2: True, source_system: 222
INFO   | uav.CameraClien | 42.631 | component.py:111 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
INFO   | uav.CameraServe | 42.633 | component.py:111 | MainThread         | Component Started self.source_component = 22, self.mav_type = 30, self.source_system = 222


set_mav_connection CameraClient mavcom.py:107 self.mav_connection = <MAVCom>
set_mav_connection CameraServer mavcom.py:107 self.mav_connection = <MAVCom>
CAMERA_Client  CAMERA_INFORMATION {time_boot_ms : 59888, vendor_name : [74, 111, 104, 110, 32, 68, 111, 101, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], model_name : [70, 97, 107, 101, 32, 67, 97, 109, 101, 114, 97, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], firmware_version : 1, focal_length : 8.0, sensor_size_h : 6.0, sensor_size_v : 4.0, resolution_h : 1920, resolution_v : 1080, lens_id : 0, flags : 0, cam_definition_version : 1, cam_definition_uri : http://example.com/camera_definition.xml, gimbal_device_id : 0} 


WARNIN | pygst.GstPipeli | 42.880 | gst_tools.py:289 | MainThread         | Gstreamer.GstPipeline: gst-core-error-quark: Pipeline construction is invalid, please add queues. (13). ../libs/gst/base/gstbasesink.c(1249): gst_base_sink_query_latency (): /GstPipeline:pipeline3/GstFPSDisplaySink:fpsdisplaysink1/GstAutoVideoSink:fps-display-video_sink/GstXvImageSink:fps-display-video_sink-actual-sink-xvimage:
Not enough buffering available for  the processing deadline of 0:00:00.015000000, add enough queues to buffer  0:00:00.015000000 additional data. Shortening processing latency to 0:00:00.000000000.


CAMERA_Client  STORAGE_INFORMATION {time_boot_ms : 59990, storage_id : 0, storage_count : 1, status : 0, total_capacity : 100000000.0, used_capacity : 0.0, available_capacity : 100000000.0, read_speed : 0.0, write_speed : 0.0, type : 0, name : } 
CAMERA_Client  CAMERA_CAPTURE_STATUS {time_boot_ms : 59991, image_status : 0, video_status : 0, image_interval : 0.0, recording_time_ms : 0, available_capacity : 0.0, image_count : 0} 
CAMERA_Client  CAMERA_INFORMATION {time_boot_ms : 59992, vendor_name : [74, 111, 104, 110, 32, 68, 111, 101, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], model_name : [70, 97, 107, 101, 32, 67, 97, 109, 101, 114, 97, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], firmware_version : 1, focal_length : 8.0, sensor_size_h : 6.0, sensor_size_v : 4.0, resolution_h : 1920, resolution_v : 1080, lens_id : 0, flags : 0, cam_definition_version : 1, cam_definition_uri : http://example.

INFO   | pygst.GstVidSrc | 44.633 | gst_tools.py:264 | MainThread         | GstVidSrcValve Shutdown requested ...
DEBUG  | pygst.GstVidSrc | 44.634 | gst_tools.py:234 | MainThread         | GstVidSrcValve Stopping pipeline ...
DEBUG  | pygst.GstVidSrc | 44.634 | gst_tools.py:238 | MainThread         | GstVidSrcValve Sending EOS event ...
DEBUG  | pygst.GstVidSrc | 44.635 | gst_tools.py:248 | MainThread         | GstVidSrcValve Reseting pipeline state ....
DEBUG  | pygst.GstVidSrc | 44.636 | gst_tools.py:255 | MainThread         | GstVidSrcValve Gst.Pipeline successfully destroyed
INFO   | pygst.GstVidSrc | 44.636 | gst_tools.py:268 | MainThread         | GstVidSrcValve successfully destroyed
INFO   | uav.GSTCamera   | 44.637 | fake_cam.py:274 | MainThread         | GSTCamera closed
INFO   | uav.CameraServe | 45.636 | component.py:366 | MainThread         | CameraServer closed
INFO   | uav.MAVCom      | 45.637 |  mavcom.py:441 | MainThread         | MAVCom  closed
INFO   | uav.CameraCli